In [ ]:
%cd ..

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import collections
import glob
import functools
import logging
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import quantities as pq

import epych
from epych.statistics import alignment

In [ ]:
%matplotlib inline

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
CONDITIONS = ["go_gloexp", "go_seqctl", "lo_gloexp", "lonaive", "lo_rndctl", "igo_seqctl"]
PRETRIAL_SECONDS = 0.5
POSTTRIAL_SECONDS = 0.5

In [ ]:
ADAPTED_ONSET = pq.Quantity(-1.0) * pq.second
ADAPTED_OFFSET = pq.Quantity(-0.5) * pq.second
ODDBALL_ONSET = pq.Quantity(-1.9017372477960602e-14) * pq.second
ODDBALL_OFFSET = pq.Quantity(0.5004545430388676) * pq.second

In [ ]:
avgs = {cond: {} for cond in CONDITIONS}

In [ ]:
area_titles = {
    "VISal": "AL",
    "VISam": "AM",
    "VISl": "LM",
    "VISp": "V1",
    "VISpm": "PM",
    "VISrl": "RL",
}
def sigtitle(name, signal):
    if name in area_titles:
        return area_titles[name]
    return name

In [ ]:
for cond in CONDITIONS:
    avgs[cond] = epych.recording.EvokedSampling.unpickle("/mnt/data/000253/grandavg_%s" % cond)
    for sig in avgs[cond].signals.values():
        sig._data = (sig.data * pq.units.volt).rescale('mV')
        sig._timestamps = sig._timestamps * pq.second
        sig._dt = sig._dt * pq.second
    logging.info("Loaded grand-averaged LFPs for %s" % cond)

In [ ]:
csds = {cond: {} for cond in CONDITIONS}

In [ ]:
for cond in CONDITIONS:
    csds[cond] = avgs[cond].smap(lambda sig: sig.downsample(4).current_source_density(depth_column="vertical"))
    for sig in csds[cond].signals.values():
        sig._data = sig.data.rescale('nA / mm**2')

In [ ]:
events = {
    "Adapted Stimulus Onset": (ADAPTED_ONSET.magnitude, "lightgreen"),
    "Adapted Stimulus Offset": (ADAPTED_OFFSET.magnitude, "red"),
    "Oddball Onset": (ODDBALL_ONSET.magnitude, 'lightgreen'),
    "Oddball Offset": (ODDBALL_OFFSET.magnitude, 'red'),
}

In [ ]:
for cond in CONDITIONS:
    avgs[cond].plot(vmin=-1e-1, vmax=1e-1, figure="mice_passiveglo_visual_%s_lfp.pdf" % cond, sigtitle=sigtitle, **events)
    csds[cond].plot(figure="mice_passiveglo_visual_%s_csd.pdf" % cond, cmap='jet_r', sigtitle=sigtitle, **events)
    logging.info("Plotted grand-averaged LFPs for condition %s" % cond)

In [ ]:
subtraction_lfps, subtraction_csds = {}, {}

In [ ]:
subtraction_events = {
    "Onset": (ODDBALL_ONSET.magnitude, 'lightgreen'),
    "Offset": (ODDBALL_OFFSET.magnitude, 'red'),
}

In [ ]:
OFFSET = pq.Quantity(0.25) * pq.second

def subtract_signals(sig):
    subtraction = sig[(ODDBALL_ONSET - OFFSET).magnitude:(ODDBALL_OFFSET + OFFSET).magnitude] - sig[(ADAPTED_ONSET - OFFSET).magnitude:(ADAPTED_OFFSET + OFFSET).magnitude]
    return subtraction.shift_timestamps(-OFFSET)

In [ ]:
for cond in CONDITIONS:
    subtraction_lfps[cond] = avgs[cond].smap(subtract_signals)
    subtraction_lfps[cond].plot(vmin=-1e-1, vmax=1e-1, figure="mice_passiveglo_visual_4minus3_%s_lfp.pdf" % cond, sigtitle=sigtitle, **subtraction_events)

    subtraction_csds[cond] = csds[cond].smap(subtract_signals)
    subtraction_csds[cond].plot(figure="mice_passiveglo_visual_4minus3_%s_csd.pdf" % cond, cmap='PiYG', sigtitle=sigtitle, **subtraction_events)
    logging.info("Plotted grand-averaged single subtractions for condition %s" % cond)

In [ ]:
CONTRASTS = [("logo", "lo_gloexp", "go_gloexp"), ("go_contrast", "go_gloexp", "go_seqctl"), ("lo_contrast", "lo_gloexp", "lo_rndctl"), ("ssa", "lo_gloexp", "igo_seqctl"), ("dd", "lo_rndctl", "lonaive")]

In [ ]:
for (name, l, r) in CONTRASTS:
    lfp = avgs[l] - avgs[r]
    lfp.plot(vmin=-1e-1, vmax=1e-1, figure="mice_passiveglo_visual_%s_lfp.pdf" % name, sigtitle=sigtitle, **events)
    csd = csds[l] - csds[r]
    csd.plot_signals(name, cmap='PiYG', sigtitle=sigtitle, **events)
    logging.info("Plotted %s contrast" % name)

    lfp = subtraction_lfps[l] - subtraction_lfps[r]
    lfp.plot(vmin=-1e-1, vmax=1e-1, figure="mice_passiveglo_visual_4minus3_%s_lfp.pdf" % name, sigtitle=sigtitle, **subtraction_events)
    csd = subtraction_csds[l] - subtraction_csds[r]
    csd.plot_signals(name + "_4minus3", cmap='PiYG', sigtitle=sigtitle, **subtraction_events)
    logging.info("Plotted contrast of %s single subtractions" % name)